# Fine-Tune Whisper For Werewolf

## Prepare Environment

In [ ]:
!pip install --quiet datasets evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.2 MB/s eta 0:00:00


In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

## Consts

In [1]:
BATCH_SIZE = 50
BOS_LEN = 2
EOS_LEN = 1
MAX_DURATION = 30
MASK_ID = -100
MAX_LENGTH = 448
SAMPLING_RATE = 16000
WORD_ERROR_PENALTY = 100

## Load Model

In [5]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration


model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

/home/alonb/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_hip.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Load Dataset

Filter: 100%|██████████| 910/910 [00:03<00:00, 261.37 examples/s]


## Prepare Dataset

In [10]:
# import numpy as np
# from datasets import load_dataset, Audio, DatasetDict, Dataset

# feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# werewolf_data = DatasetDict()

# werewolf_data["train"] = load_dataset("parquet", data_files=["https://huggingface.co/datasets/iohadrubin/werewolf_dialogue_data_10sec/resolve/main/data/train-00002-of-00014-e0e6b0000eedceb4.parquet"])["train"]
# werewolf_data["test"] = load_dataset("parquet", data_files=["https://huggingface.co/datasets/iohadrubin/werewolf_dialogue_data_10sec/resolve/main/data/train-00009-of-00014-603088eeb6352a9b.parquet"])["train"]

# werewolf_data = werewolf_data.filter(lambda x: x["end"] - x["start"] <= MAX_DURATION and x["dialogue"][-1]["target"] is not None)
# werewolf_data = werewolf_data.remove_columns(["start", "end", "idx", "Game_ID", "file_name", "video_name", "startRoles", "startTime", "endRoles", "playerNames"])

# prompt_prefix, prompt_suffix = """Given the following dialogue and audio, assign the last utterance one or more of the following tags (delimited by commas):
# 'Accusation', 'Defense', 'Evidence', 'Identity Declaration', 'Interrogation', 'No Strategy'

# ```
# """, """
# ```

# Reminder - Assign one or more of the following tags to the last utterance (delimited by commas):
# 'Accusation', 'Defense', 'Evidence', 'Identity Declaration', 'Interrogation', 'No Strategy'

# Assignment:
# """

# prompt_prefix_len = len(tokenizer.encode(prompt_prefix)) - EOS_LEN
# max_prompt_len = MAX_LENGTH - prompt_prefix_len

# def prepare_decoder_input_ids_and_labels(sample):
#   dialogue = "\n".join(f"{x['speaker']}: {x['utterance']}" for x in sample["dialogue"])
#   target = sample["dialogue"][-1]["target"]
#   text = prompt_prefix + dialogue + prompt_suffix + target

#   input_ids = tokenizer.encode(text)
#   if len(input_ids) > MAX_LENGTH:
#     input_ids = input_ids[:prompt_prefix_len] + input_ids[-max_prompt_len:]

#   decoder_input_ids = np.array(input_ids)
#   sample["decoder_input_ids"] = decoder_input_ids

#   labels = np.array(input_ids)
#   target_len = len(tokenizer.encode(target)) - BOS_LEN
#   labels[:-target_len] = MASK_ID
#   sample["labels"] = labels

#   return sample

# def prepare_audio(batch):
#   audio_arrays = [x["array"] for x in batch["audio"]]

#   batch["input_features"] = feature_extractor(audio_arrays, sampling_rate=SAMPLING_RATE, return_tensors="pt").input_features

#   return batch


# werewolf_data = werewolf_data.map(prepare_decoder_input_ids_and_labels)
# werewolf_data = werewolf_data.map(prepare_audio, batched=True, batch_size=BATCH_SIZE)

Map:   0%|          | 0/901 [00:00<?, ? examples/s]2025-02-25 13:57:49.525184: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 13:57:49.529872: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 13:57:49.540949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740491869.559348 3963623 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740491869.564317 3963623 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 13:57:49.583691: I tensorflow/core/platform/cpu_feature_guard.cc:210] This Tens

## Data Collator

In [1]:
from src.audio.data import load_and_prepare_werewolf_data

/home/alonb/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_hip.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
werewolf_data, tokenizer, feature_extractor = load_and_prepare_werewolf_data("openai/whisper-small")

Map:   0%|          | 0/12504 [00:00<?, ? examples/s]

2025-02-25 14:47:17.143015: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 14:47:17.147260: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 14:47:17.159106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740494837.180113 3969732 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740494837.186571 3969732 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 14:47:17.208288: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [21]:
werewolf_data.set_format(type="numpy", columns=["input_features", "decoder_input_ids", "labels"])

In [39]:
itr = iter(werewolf_data["train"].iter(16))

In [45]:
batch = next(itr)
jax.tree.map(np.shape, batch)

{'decoder_input_ids': (16,), 'input_features': (16, 80, 3000), 'labels': (16,)}

In [26]:
import jax.numpy as jnp
import jax

In [49]:

# batch


from dataclasses import dataclass
from transformers import WhisperProcessor


data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)


In [50]:
data_collator(batch)

{'input_features': tensor([[[ 1.4271e-01,  3.1942e-01,  3.2700e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         [ 1.8945e-01,  2.9090e-01,  3.5406e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         [ 2.3707e-01,  1.2807e-01,  2.5274e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         ...,
         [-6.8846e-01, -6.7930e-01, -6.8846e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         [-6.8846e-01, -6.7836e-01, -6.8846e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         [-6.8143e-01, -6.3424e-01, -6.7985e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01]],

        [[ 2.4604e-01,  3.1551e-01,  3.2865e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         [ 9.7445e-02,  3.2855e-01,  3.2837e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         [-4.9492e-02,  2.2324e-01,  1.8133e-01,  ..., -6.8846e-01,
          -6.8846e-01, -6.8846e-01],
         ...,
      

## Compute Metrics

In [53]:
!python3.10 -m pip install --upgrade evaluate jiwer

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 87.9 MB/s eta 0:00:00


In [55]:
!python3.10 -m  pip install  evaluate==0.4.3

Defaulting to user installation because normal site-packages is not writeable


In [54]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == MASK_ID] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = WORD_ERROR_PENALTY * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

AttributeError: 'DownloadConfig' object has no attribute 'use_auth_token'

## Training Arguments

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-werewolf",
    eval_strategy="steps",
    eval_steps=1000,
    max_steps=4000,
    warmup_steps=500,
    logging_steps=25,
    save_steps=1000,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to=["tensorboard"],
    push_to_hub=True,
    gradient_checkpointing=True,
    predict_with_generate=True
)

## Trainer

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=werewolf_data["train"],
    eval_dataset=werewolf_data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

### Training

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ValueError: one or more references are empty strings

## Publish

In [ ]:
kwargs = {
    "dataset_tags": "iohadrubin/werewolf_dialogue_data_10sec",
    "dataset_args": "split: test",
    "model_name": "Whisper Small Werewolf",
    "finetuned_from": "openai/whisper-small",
    "tasks": "classification",
}

In [ ]:
trainer.push_to_hub(**kwargs)